In [1]:
import pandas as pd
import numpy as np
import json as js

In [2]:
temp_quarter_df = pd.read_csv('../SourceMaterial/Measures/Results/water_result.csv')
conductivity_quarter_df = pd.read_csv('../SourceMaterial/Measures/Results/conductivity_result.csv')
ph_quarter_df = pd.read_csv('../SourceMaterial/Measures/Results/ph_result.csv')

df_list = [temp_quarter_df, conductivity_quarter_df, ph_quarter_df]
name_list = ['temp', 'conductivity', 'ph']
quarter_list = df_list[0]['q'].unique()

In [3]:
temp_quarter_df.head()

,Unnamed: 0,huc,quarter,q,year,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,0,Totuskey Creek,Q1 2012,Q1,2012,18.940000,4.352011,9.100000,16.0,2.5,37,2019-12-19,2012-01-05
1,1,Totuskey Creek,Q2 2012,Q2,2012,5.062500,2.250000,17.250000,21.0,14.0,37,2019-12-19,2012-01-05
2,2,Totuskey Creek,Q3 2012,Q3,2012,4.888889,2.211083,23.166667,25.0,18.5,37,2019-12-19,2012-01-05
3,3,Totuskey Creek,Q4 2012,Q4,2012,20.333333,4.509250,11.500000,20.5,7.0,37,2019-12-19,2012-01-05
4,4,Totuskey Creek,Q1 2013,Q1,2013,1.426649,1.194424,5.645833,9.0,4.5,37,2019-12-19,2012-01-05


In [4]:
conductivity_quarter_df.head()

,Unnamed: 0,huc,quarter,q,year,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,0,Muddy Run-Susquehanna River,Q2 2015,Q2,2015,0.000000,0.000000,254.000000,254.0,254.0,91,2019-07-10,2015-05-09
1,1,Muddy Run-Susquehanna River,Q3 2015,Q3,2015,1350.888889,36.754440,272.333333,318.0,228.0,91,2019-07-10,2015-05-09
2,2,Muddy Run-Susquehanna River,Q4 2015,Q4,2015,729.000000,27.000000,271.000000,298.0,244.0,91,2019-07-10,2015-05-09
3,3,Muddy Run-Susquehanna River,Q1 2016,Q1,2016,144.666667,12.027746,222.000000,239.0,213.0,91,2019-07-10,2015-05-09
4,4,Muddy Run-Susquehanna River,Q2 2016,Q2,2016,116.168889,10.778167,206.866667,220.0,193.6,91,2019-07-10,2015-05-09


In [5]:
# check each df for huc names, if a huc appears in one df but not another, drop that huc from the source df
# sort hucs
all_hucs = []
suitable_hucs = []
bad_hucs = []

for df in df_list:
    hucs = df['huc'].unique()
    for huc in hucs:
        all_hucs.append(huc)
for huc in all_hucs:
    if all_hucs.count(huc) == 3:
        if huc not in suitable_hucs:
            suitable_hucs.append(huc)
    elif all_hucs.count(huc) < 3:
        if huc not in bad_hucs:
            bad_hucs.append(huc)

for df in df_list:
    df = df[df['huc'].isin(suitable_hucs)]

In [6]:
print(f"Found {len(suitable_hucs)} suitable hucs, and {len(bad_hucs)} un-usable hucs")

Found 29 suitable hucs, and 17 un-usable hucs


In [7]:
result_list = []
index = 0
# for each measure
for df in df_list:
    avg_over_time_list = []
     # for each suitable huc
    for huc in suitable_hucs: 
        # for each quarter
        huc_df = df.loc[df['huc'] == huc]
        for quarter in quarter_list:
            # find avg change over time
            working_df = huc_df.loc[df['q'] == quarter]
            if index == 1:
                print(quarter)
            earliest_avg = working_df['avg'].loc[working_df['year']==working_df['year'].min()].values[0]
            final_avg = working_df['avg'].loc[working_df['year']==working_df['year'].max()].values[0]
            percent_change = ((final_avg - earliest_avg)/earliest_avg)*100 
            lowest_avg = working_df['avg'].min()
            highest_avg = working_df['avg'].max()
            q_avg_result = {
                'huc': huc,
                'quarter': quarter,
                'percent_change': percent_change,
                'highest_avg': highest_avg,
                'lowest_avg': lowest_avg,
                'start': working_df['year'].min(),
                'end': working_df['year'].max()
            }
            avg_over_time_list.append(q_avg_result)
    result_df = pd.DataFrame(avg_over_time_list)
    result_df.to_csv(f"../SourceMaterial/Measures/Results/Final_Analytics/{name_list[index]}_percent_change.csv")
    result_list.append(result_df)
    index+=1

Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4
Q1
Q2
Q3
Q4


In [8]:
result_list[0]

,huc,quarter,percent_change,highest_avg,lowest_avg,start,end
0,Upham Brook,Q1,-30.421456,14.500000,8.033333,2012,2020
1,Upham Brook,Q2,7.710843,22.350000,17.000000,2011,2020
2,Upham Brook,Q3,8.218623,24.980000,21.600000,2011,2019
3,Upham Brook,Q4,3.609821,14.940000,11.875000,2011,2019
4,Lower Nanticoke Creek,Q1,-217.370892,5.000000,-1.666667,2013,2019
...,...,...,...,...,...,...,...
111,Muddy Run-Susquehanna River,Q4,-14.651163,10.750000,7.633333,2015,2018
112,Fishing Creek-Susquehanna River,Q1,-21.844725,9.133333,2.823077,2016,2019
113,Fishing Creek-Susquehanna River,Q2,-7.084189,15.014737,13.575000,2016,2019
114,Fishing Creek-Susquehanna River,Q3,-5.161549,19.836364,18.300000,2016,2019


In [9]:
result_list[1]

,huc,quarter,percent_change,highest_avg,lowest_avg,start,end
0,Upham Brook,Q1,-37.791667,2610.000000,34.916667,2012,2020
1,Upham Brook,Q2,632.043565,1646.000000,169.303226,2011,2020
2,Upham Brook,Q3,38.384000,365.116279,103.642857,2011,2019
3,Upham Brook,Q4,-88.608997,1275.000000,79.125000,2011,2019
4,Lower Nanticoke Creek,Q1,9.320930,311.000000,172.500000,2013,2019
...,...,...,...,...,...,...,...
111,Muddy Run-Susquehanna River,Q4,-21.586716,271.000000,212.500000,2015,2018
112,Fishing Creek-Susquehanna River,Q1,18.587662,265.636364,224.000000,2016,2019
113,Fishing Creek-Susquehanna River,Q2,21.744708,255.857143,194.875000,2016,2019
114,Fishing Creek-Susquehanna River,Q3,6.232160,256.076923,238.863636,2016,2019


In [10]:
result_list[2]

,huc,quarter,percent_change,highest_avg,lowest_avg,start,end
0,Upham Brook,Q1,21.378941,7.570000,6.236667,2012,2020
1,Upham Brook,Q2,2.398435,7.297857,6.595000,2011,2019
2,Upham Brook,Q3,0.367002,7.224286,6.650000,2011,2019
3,Upham Brook,Q4,2.736752,7.402222,6.620000,2011,2019
4,Lower Nanticoke Creek,Q1,0.000000,7.000000,6.750000,2013,2019
...,...,...,...,...,...,...,...
111,Muddy Run-Susquehanna River,Q4,-4.346467,8.566667,7.197143,2015,2018
112,Fishing Creek-Susquehanna River,Q1,-15.548164,8.836667,7.462727,2016,2019
113,Fishing Creek-Susquehanna River,Q2,-12.181045,8.230000,7.147143,2016,2019
114,Fishing Creek-Susquehanna River,Q3,-12.230115,8.126364,7.132500,2016,2019


In [11]:
ph_quarter_df.head()

,Unnamed: 0,huc,quarter,q,year,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,0,Muddy Run-Susquehanna River,Q2 2015,Q2,2015,0.000000,0.000000,6.550000,6.55,6.55,91,2019-07-10,2015-05-09
1,1,Muddy Run-Susquehanna River,Q3 2015,Q3,2015,0.645000,0.803119,8.580000,9.63,7.68,91,2019-07-10,2015-05-09
2,2,Muddy Run-Susquehanna River,Q4 2015,Q4,2015,0.030625,0.175000,7.995000,8.17,7.82,91,2019-07-10,2015-05-09
3,3,Muddy Run-Susquehanna River,Q1 2016,Q1,2016,0.105089,0.324174,7.943333,8.40,7.68,91,2019-07-10,2015-05-09
4,4,Muddy Run-Susquehanna River,Q2 2016,Q2,2016,0.005956,0.077172,7.876667,7.95,7.77,91,2019-07-10,2015-05-09


In [12]:
temp_quarter_df.head()

,Unnamed: 0,huc,quarter,q,year,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,0,Totuskey Creek,Q1 2012,Q1,2012,18.940000,4.352011,9.100000,16.0,2.5,37,2019-12-19,2012-01-05
1,1,Totuskey Creek,Q2 2012,Q2,2012,5.062500,2.250000,17.250000,21.0,14.0,37,2019-12-19,2012-01-05
2,2,Totuskey Creek,Q3 2012,Q3,2012,4.888889,2.211083,23.166667,25.0,18.5,37,2019-12-19,2012-01-05
3,3,Totuskey Creek,Q4 2012,Q4,2012,20.333333,4.509250,11.500000,20.5,7.0,37,2019-12-19,2012-01-05
4,4,Totuskey Creek,Q1 2013,Q1,2013,1.426649,1.194424,5.645833,9.0,4.5,37,2019-12-19,2012-01-05
